## Feature Engineering

In [1]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

SEED = 42

In [2]:
# Data import
load_dotenv()

DATA_PATH = Path(os.getenv('PROJECT_ROOT'), '.') / 'data'
TRACKS_FILE = DATA_PATH / 'spotify_track_genres_preprocessed.csv'

tracks = pd.read_csv(TRACKS_FILE)

In [4]:
# Check data
tracks.info()
tracks.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10416 entries, 0 to 10415
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          10416 non-null  object 
 1   duration_ms       10416 non-null  int64  
 2   track_name        10416 non-null  object 
 3   track_popularity  10416 non-null  int64  
 4   acousticness      10416 non-null  float64
 5   danceability      10416 non-null  float64
 6   energy            10416 non-null  float64
 7   instrumentalness  10416 non-null  float64
 8   key               10416 non-null  int64  
 9   liveness          10416 non-null  float64
 10  loudness          10416 non-null  float64
 11  mode              10416 non-null  int64  
 12  speechiness       10416 non-null  float64
 13  tempo             10416 non-null  float64
 14  time_signature    10416 non-null  int64  
 15  valence           10416 non-null  float64
 16  artist_ids        10416 non-null  object

,duration_ms,track_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,n_artists,n_genres
count,1.041600e+04,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000,10416.000000
mean,2.286264e+05,52.253744,0.360780,0.577236,0.564434,0.205984,5.245776,0.174369,-9.679182,0.612903,0.086908,117.952822,3.886425,0.473638,1.384121,4.404570
std,1.056449e+05,24.445176,0.353521,0.194247,0.275336,0.349380,3.554538,0.139941,6.641682,0.487109,0.088110,30.271137,0.439159,0.264467,0.787452,3.528924
min,2.560000e+04,0.000000,0.000002,0.000000,0.001100,0.000000,0.000000,0.015700,-47.903000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,1.736338e+05,40.000000,0.036650,0.452000,0.364000,0.000000,2.000000,0.093100,-11.953000,0.000000,0.036800,94.195500,4.000000,0.249000,1.000000,2.000000
50%,2.135395e+05,56.000000,0.216000,0.596000,0.603000,0.000190,5.000000,0.119000,-7.617500,1.000000,0.049000,118.005000,4.000000,0.472000,1.000000,4.000000
75%,2.584020e+05,71.000000,0.699000,0.726000,0.791000,0.266250,8.000000,0.206000,-5.264750,1.000000,0.091000,137.644500,4.000000,0.688000,2.000000,6.000000
max,2.668693e+06,100.000000,0.996000,0.979000,0.999000,0.994000,11.000000,1.000000,2.015000,1.000000,0.937000,217.969000,5.000000,0.988000,16.000000,91.000000


In [5]:
tracks.head()

,track_id,duration_ms,track_name,track_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_ids,n_artists,artist_genres,n_genres
0,000u1dTg7y1XCDXi80hbBX,300600,I Don't Want A Lover,1,0.196000,0.756,0.470,0.000487,7,0.1260,-12.615,1,0.0394,120.484,4,0.4300,['5JsdVATHNPE0XdMFMRoSuf'],1,"['scottish rock', 'new wave pop', 'new romantic']",3
1,003FTlCpBTM4eSqYSWPv4H,233266,"Swing, Swing",70,0.000065,0.553,0.717,0.128000,7,0.0385,-5.857,1,0.0318,127.947,4,0.3180,['3vAaWhdBR38Q02ohXqaNHT'],1,"['neon pop punk', 'neo mellow', 'post-grunge',...",7
2,003vvx7Niy0yvhvHt4a68B,222973,Mr. Brightside,90,0.001210,0.352,0.911,0.000000,1,0.0995,-5.230,1,0.0747,148.033,4,0.2360,['0C0XlULifJtAgn6ZNCW2eu'],1,"['permanent wave', 'alternative rock', 'dance ...",5
3,008RKiNmjW5Lb6Ocumq6MA,298666,TWO OF US,46,0.927000,0.144,0.188,0.684000,4,0.1520,-13.680,0,0.0399,85.762,4,0.0743,['6mOCYCpXBAmulHN4kMfQaa'],1,['anime score'],1
4,009ImBOrIUlWgla8U05RAC,228878,Unconditionally,82,0.002730,0.432,0.725,0.000000,7,0.2080,-4.862,1,0.0431,128.902,4,0.3530,['6jJ0s89eD6GaHleKKya26X'],1,['pop'],1
